In [15]:
import os
import leafmap
from samgeo import SamGeo, tms_to_geotiff, get_basemaps
import pickle
import geopandas as gpd
from shapely.geometry import box

In [10]:
import yaml
config_path = '/home/tu/tu_tu/tu_zxmav84/DS_Project/modules/config.yml'
with open(config_path, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
city_3d_model_path = config['data']['city_3d_model']
ortho_path = config['data']['orthophotos']
rooftop_tifs_path = config['data']['rooftop_tifs']

### Import 3D City model data

In [16]:
with open(city_3d_model_path + 'processed/processed_roofs.pkl', 'rb') as f:
    gdf = pickle.load(f)

# Subset to only flat roofs
gdf = gdf[gdf.roofType.isin([1000])]
# Subset to only inner city
bbox = box(*[11.557617,48.123591,11.599846,48.147649])
gdf = gdf[gdf.geometry_4326.intersects(bbox)]
# Use only buildings with height > 3 meter to get rid of small objects like bus stops
#gdf = gdf[gdf.measuredHeight > 3]
# Get rid of bridges and Überdachungen
gdf = gdf[gdf.function.isin(['51009_1610', '53001_1800']) == False]

# Susbet to relevant columns
gdf = gdf[['id', 'geometry']]

# Sort by area of roof 
gdf['area'] = gdf.geometry.area
gdf.sort_values('area', ascending=False, inplace=True)


gdf.head(3)

,id,geometry,area
3070,DEBY_LOD2_4955860,"MULTIPOLYGON (((692064.360 5334089.410, 692064...",12072.216919
3329,DEBY_LOD2_4958619,"MULTIPOLYGON (((691399.952 5335853.259, 691397...",8572.973060
3898,DEBY_LOD2_5007168,"MULTIPOLYGON (((692435.339 5335549.354, 692428...",8416.904426


In [31]:
gdf.geometry = gdf.geometry.to_crs('EPSG:4326')
bbox = list(gdf.iloc[2000,:].geometry.bounds)
bbox

[11.594285120585656,
 48.125770574044374,
 11.594458336977054,
 48.125879243763386]

In [30]:
bbox = [11.558889,48.139411,11.560644,48.141230]

In [8]:
image = "satellite.tif"

In [39]:
tms_to_geotiff(output=image, bbox=bbox, zoom=21, source="Satellite", overwrite=True)


Downloaded image 1/2
Downloaded image 2/2
Saving GeoTIFF. Please wait...
Image saved to satellite.tif
